# VencoPy Tutorial 5

This tutorial aims to give a more in depth overview into the flexEstimator class and showcases some features that can be customised.

In [ ]:
import sys
import os
from os import path
import pandas as pd
from pathlib import Path
import pathlib
#from ruamel.yaml import YAML

sys.path.append(path.dirname(path.dirname(path.dirname(Path.cwd()))))

from vencopy.classes.dataParsers import DataParser
from vencopy.classes.tripDiaryBuilders import TripDiaryBuilder
from vencopy.classes.gridModelers import GridModeler
from vencopy.classes.flexEstimators import FlexEstimator
from vencopy.classes.evaluators import Evaluator
from vencopy.scripts.globalFunctions import loadConfigDict, createOutputFolders

print("Current working directory: {0}".format(os.getcwd()))


In [ ]:
configNames = ('globalConfig', 'localPathConfig', 'parseConfig', 'tripConfig', 'gridConfig', 'flexConfig', 'evaluatorConfig')
basePath = Path.cwd().parent.parent
configDict = loadConfigDict(configNames, basePath)
createOutputFolders(configDict=configDict)

# Adapt relative paths in config for tutorials
configDict['globalConfig']['pathRelative']['plots'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['plots']
configDict['globalConfig']['pathRelative']['parseOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['parseOutput']
configDict['globalConfig']['pathRelative']['diaryOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['diaryOutput']
configDict['globalConfig']['pathRelative']['gridOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['gridOutput']
configDict['globalConfig']['pathRelative']['flexOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['flexOutput']
configDict['globalConfig']['pathRelative']['evalOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['evalOutput']

# Set reference dataset
datasetID = 'MiD17'

# Modify the localPathConfig file to point to the .csv file in the sampling folder in the tutorials directory where the dataset for the tutorials lies.
configDict['localPathConfig']['pathAbsolute'][datasetID] = pathlib.Path.cwd().parent / 'data_sampling'

# Similarly we modify the datasetID in the global config file
configDict['globalConfig']['files'][datasetID]['tripsDataRaw'] = datasetID + '.csv'

# We also modify the parseConfig by removing some of the columns that are normally parsed from the MiD, which are not available in our semplified test dataframe
del configDict['parseConfig']['dataVariables']['hhID']
del configDict['parseConfig']['dataVariables']['personID']


## FlexEstimator config file

The FlexEstimator config file contains the technical specifications..

In [ ]:
configDict['flexConfig']['inputDataScalars'][datasetID]

## _FlexEstimator_ class

To use the FlexEstimator class, we first need to run the DataParses as this is an argument to the FlexEstimator class.

In [ ]:
vpData = DataParser(datasetID=datasetID, configDict=configDict, loadEncrypted=False)
vpData.process()

In [ ]:
# We can check how trips are distributed based on their length in km
%matplotlib inline
vpData.data.tripDistance.plot.hist() 

In [ ]:
# Estimate charging flexibility based on driving profiles and charge connection
vpFlex = FlexEstimator(configDict=configDict, datasetID=datasetID, ParseData=vpData)
vpFlex.baseProfileCalculation()
vpFlex.filter()
vpFlex.aggregate()
vpFlex.correct()
vpFlex.normalize()
vpFlex.writeOut()

In [ ]:
vpEval = Evaluator(configDict=configDict, parseData=pd.Series(data=vpData, index=[datasetID]))
vpEval.plotProfiles(flexEstimator=vpFlex)

To analyse its influence on the demand-side flexibility from EV, we will though charge the assumed size of the battery from 50 kWh to 75 kWh.

In [ ]:
configDict['flexConfig']['inputDataScalars'][datasetID]['Battery_capacity'] = 100.0
#configDict['flexConfig']['inputDataScalars'][datasetID]['Electric_consumption'] = 20.0

configDict['flexConfig']['inputDataScalars'][datasetID]

In [ ]:
# Estimate charging flexibility based on driving profiles and charge connection
vpFlex = FlexEstimator(configDict=configDict, datasetID=datasetID, ParseData=vpData)
vpFlex.baseProfileCalculation()
vpFlex.filter()
vpFlex.aggregate()
vpFlex.correct()
vpFlex.normalize()
vpFlex.writeOut()

In [ ]:
vpEval = Evaluator(configDict=configDict, parseData=pd.Series(data=vpData, index=[datasetID]))
vpEval.plotProfiles(flexEstimator=vpFlex)

By doubling the battery capacity we cannot really see from the graphs distinctive changes but we see that, as a result, we have 2 more profiles considered.

## Next Steps

Come back, there will be more upcoming tutorials! :)